In [1]:
import os
import requests
from bs4 import BeautifulSoup

BASE_URL = "https://www.kplc.co.ke/customer-support"
SAVE_DIR = "kplc_pdfs"

os.makedirs(SAVE_DIR, exist_ok=True)

def get_pdf_links(page):
    url = f"{BASE_URL}?page={page}#powerschedule"
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    
    pdf_links = []
    for a in soup.find_all("a", href=True):
        href = a["href"]
        if href.endswith(".pdf") and "storage" in href:
            pdf_links.append(href)
    return pdf_links

def download_pdf(url, save_dir=SAVE_DIR):
    filename = url.split("/")[-1]
    filepath = os.path.join(save_dir, filename)
    if not os.path.exists(filepath):  # avoid duplicates
        print(f"Downloading: {filename}")
        r = requests.get(url)
        with open(filepath, "wb") as f:
            f.write(r.content)

# Loop through the 7 pages
all_pdfs = []
for page in range(1, 8):
    links = get_pdf_links(page)
    all_pdfs.extend(links)

# Remove duplicates just in case
all_pdfs = list(set(all_pdfs))

# Download PDFs
for pdf in all_pdfs:
    download_pdf(pdf)


Downloading: Kenya_Power_Sustainability_Strategy_2024_18th_Nov.pdf
Downloading: 01J8PJD2N8FWYSXJ3AD11TQH8C.pdf
Downloading: 01JYA1DANZCTPZA6ZC6HMHVKDD.pdf
Downloading: 01J7GBQNFHE57VDQ3VE5P79VM5.pdf
Downloading: 01JMHWRB64HYFS4DXNT8G2AXPS.pdf
Downloading: Customer_Complaint_Policy_and_Process_7_in_x_9.5_in_final.pdf
Downloading: 01JSK6XSV27X6YHDX9CVZZ746Z.pdf
Downloading: 01JTR3EEJMZGWYZXDV0TXP9XH0.pdf
Downloading: 01JWE3DAJTH2DBTJK6BZGEZDK9.pdf
Downloading: 01JVY1ETS6ASGH434Y6NRZREKP.pdf
Downloading: 01JP7XEH05QF9NJM4TGRY2RMTY.pdf
Downloading: 01JGNPY56W5V0NXNDG3Q6JGKRK.pdf
Downloading: 01K2P78RAN74262NZS75AAQSVC.pdf
Downloading: 01K4CVT9654ZXNHZ7PW0Y3AH3K.pdf
Downloading: 01J871DG040Y12R55BTGT09WYN.pdf
Downloading: 01J1FA2JXK3QNX0PM6YAP0P3T2.pdf
Downloading: 01K1JKNRVT8MN93RGC2KPH6248.pdf
Downloading: 01JF74GM6PZ4W0V431G0CW2J28.pdf
Downloading: 01JDSFBFT07Q0ARPMF06P19WF7.pdf
Downloading: 01JC3E4SK6Z5PR3C5XKKSFVXW1.pdf
Downloading: 01JZ7KGAR4GPMNPMRCM5P1M9ST.pdf
Downloading: Final_E_M

In [2]:
len(all_pdfs)

61

# Extract pdf text

In [3]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.4 MB/s eta 0:00:00


In [4]:
import PyPDF2

# Open the PDF
with open("/kaggle/working/kplc_pdfs/01J1FA2JXK3QNX0PM6YAP0P3T2.pdf", "rb") as f:
    reader = PyPDF2.PdfReader(f)
    print(f"Number of pages: {len(reader.pages)}")
    text = ''
    for j in range(len(reader.pages)):
        # Read page
        page = reader.pages[j]
        text += page.extract_text()
        # print(text)
        #break
print(text)

Number of pages: 2
                                                                                                         Interruption of  
Electricity Supply   
Notice is hereby given under R ule 27 of the Electric Power Rules  
That the electricity supply will be interrupted as here under:  
(It is necessary to interrupt supply periodically in order to facilitate 
maintenance and upgrade of power lines to the network; to connect new 
customers or to replace power lines during road construction, etc.)  
 
NAIROBI REGION  
AREA: PART OF EASTLEIGH  
DATE: Sunday 30.06.2024                                             TIME: 9.00 A.M. – 5.00 P.M.  
Mohammed Yusuf Haji Ave, I smail Hassan, Moyale Mall, Sirare, Taran, Pumwani 
Hosp, California, Care Hosp, Mabruk & adjacent customers.  
 
AREA: KYANGOMBE, SYOKIMAU  
DATE: Sunday 30.06.2024                                             TIME: 9.00 A.M.  – 5.00 P.M.  
Inland Container Deport (I CD), Kyang’ombe, Heavy Engineering, Tile & 
Carpet,

In [5]:
import re

pattern = re.compile(
    r'(?:PARTS OF\s+(?P<county>[A-Z\s]+?)\s+COUNTY)?'   # optional county
    r'.*?AREA:\s*(?P<area>.*?)\s*'                      # area
    r'DATE:\s*(?P<date>.*?)\s*'                         # date
    r'TIME:?\s*(?P<time>[0-9A\.M\-–\s:]+)',             # time
    re.DOTALL
)

matches = pattern.finditer(text)
for m in matches:
    print(m.groupdict())


{'county': None, 'area': 'PART OF EASTLEIGH', 'date': 'Sunday 30.06.2024', 'time': '9.00 A.M. – 5.00 '}
{'county': None, 'area': 'KYANGOMBE, SYOKIMAU', 'date': 'Sunday 30.06.2024', 'time': '9.00 A.M.  – 5.00 '}
{'county': None, 'area': 'PART OF EASTLEIGH', 'date': 'Wednesday 03.07.2024', 'time': '9.00 A.M. – 5.00 '}
{'county': None, 'area': 'VILLAGE MARKET', 'date': 'Thursday 04.07.2024', 'time': ': 9.00 A.M. – 5.00 '}
{'county': None, 'area': 'EMALI, SULTAN', 'date': 'Thursday  04.07.2024', 'time': '9.00 A.M. – 5.00 '}
{'county': None, 'area': 'PART OF MOMBASA ROAD, NGUU', 'date': 'Thursday  04.07.2024', 'time': '9.00 A.M. – 5.00 '}
{'county': None, 'area': 'KISERIAN', 'date': 'Sunday 30.06.2024', 'time': '9.00 A.M. – 3.00 '}
{'county': None, 'area': 'KAPKOROS, KIPLOKYI', 'date': 'Monday 01.07.2024', 'time': '9.00 A.M. - 4.00 '}
{'county': None, 'area': 'SINENDET CENTRE', 'date': 'Saturday 29.06.2024', 'time': '10.00 A.M. – 5.00 '}
{'county': None, 'area': 'KIPKORGOT, CARBACID', 'date

In [6]:
import re
import pandas as pd

text = text#/"""..."""  # your notice

records = []
county = None

for line in text.splitlines():
    line = line.strip()

    # detect county
    if "PARTS OF" in line and "COUNTY" in line:
        county = line.replace("PARTS OF", "").replace("COUNTY", "").strip()
    if "NAIROBI REGION" in line:
        county="Nairobi"
    # detect area/date/time
    if line.startswith("AREA:"):
        area = line.replace("AREA:", "").strip()
    if line.startswith("AREA :"):
        area = line.replace("AREA :", "").strip()
    elif line.startswith("DATE:"):
        date = re.search(r"DATE:\s*(.*?)\s{2,}", line)
        time = re.search(r"TIME:?\s*(.*)", line)
        if date and time:
            records.append({
                "county": county,
                "area": area,
                "date": date.group(1).strip(),
                "time": time.group(1).strip()
            })

df = pd.DataFrame(records)
print(df.head(30))


           county                            area                  date  \
0         Nairobi               PART OF EASTLEIGH     Sunday 30.06.2024   
1         Nairobi             KYANGOMBE, SYOKIMAU     Sunday 30.06.2024   
2         Nairobi               PART OF EASTLEIGH  Wednesday 03.07.2024   
3         Nairobi                  VILLAGE MARKET   Thursday 04.07.2024   
4         MAKEUNI                   EMALI, SULTAN              Thursday   
5         MAKEUNI      PART OF MOMBASA ROAD, NGUU              Thursday   
6         KAJIADO                        KISERIAN     Sunday 30.06.2024   
7           BOMET              KAPKOROS, KIPLOKYI     Monday 01.07.2024   
8           BOMET                 SINENDET CENTRE   Saturday 29.06.2024   
9           BOMET             KIPKORGOT, CARBACID    Tuesday 02.07.2024   
10    TRANS NZOIA         KITALE TOWN, MARAMBACHI   Saturday 29.06.2024   
11         KISUMU                           CHIGA  Wednesday 03.07.2024   
12          BUSIA        

In [7]:
import re
import pandas as pd

# text = """..."""  # your full notice

records = []
county, area, date, time, places = None, None, None, None, []

lines = text.splitlines()
i = 0
while i < len(lines):
    line = lines[i].strip()

    # Detect county
    if "PARTS OF" in line and "COUNTY" in line:
        county = line.replace("PARTS OF", "").replace("COUNTY", "").strip()

    # Detect area
    if line.startswith("AREA:"):
        area = line.replace("AREA:", "").strip()
        # Look ahead for DATE + TIME on same or next line
        j = i+1
        while j < len(lines) and not lines[j].strip().startswith("AREA:") \
                             and "DATE:" not in lines[j]:
            j += 1
        if j < len(lines):
            date_match = re.search(r"DATE:\s*([^\t]+?)\s+TIME", lines[j])
            time_match = re.search(r"TIME:?\s*(.*)", lines[j])
            if date_match and time_match:
                date = date_match.group(1).strip()
                time = time_match.group(1).strip()
                # Collect places until next AREA/COUNTY/REGION header
                places_text = []
                k = j+1
                while k < len(lines) and not lines[k].startswith("AREA:") \
                                     and "PARTS OF" not in lines[k] \
                                     and "REGION" not in lines[k]:
                    places_text.append(lines[k].strip())
                    k += 1
                # Split into clean list
                places = re.split(r",|&", " ".join(places_text))
                places = [p.strip() for p in places if p.strip() and "adjacent" not in p.lower()]
                records.append({
                    "county": county,
                    "area": area,
                    "date": date,
                    "time": time,
                    "places": places
                })
        i = j  # jump ahead
    i += 1

df = pd.DataFrame(records)
print(df.head(10))


    county                        area                  date  \
0     None           PART OF EASTLEIGH     Sunday 30.06.2024   
1     None         KYANGOMBE, SYOKIMAU     Sunday 30.06.2024   
2     None           PART OF EASTLEIGH  Wednesday 03.07.2024   
3     None              VILLAGE MARKET   Thursday 04.07.2024   
4  MAKEUNI               EMALI, SULTAN  Thursday  04.07.2024   
5  MAKEUNI  PART OF MOMBASA ROAD, NGUU  Thursday  04.07.2024   
6  KAJIADO                    KISERIAN     Sunday 30.06.2024   
7    BOMET          KAPKOROS, KIPLOKYI     Monday 01.07.2024   
8    BOMET             SINENDET CENTRE   Saturday 29.06.2024   
9    BOMET         KIPKORGOT, CARBACID    Tuesday 02.07.2024   

                      time                                             places  
0    9.00 A.M. – 5.00 P.M.  [Mohammed Yusuf Haji Ave, I smail Hassan, Moya...  
1   9.00 A.M.  – 5.00 P.M.  [Inland Container Deport (I CD), Kyang’ombe, H...  
2    9.00 A.M. – 5.00 P.M.  [Starehe  Boys Centre, Pumw

In [8]:
import re
import json
import csv

def parse_schedule(raw_text):
    # Clean spacing
    text = re.sub(r"\s+", " ", raw_text)

    # Find all REGION/COUNTY headers (all caps words ending with REGION/COUNTY)
    headers = re.finditer(r"([A-Z\s]+(?:REGION|COUNTY))", text)
    header_positions = [(m.start(), m.group().strip()) for m in headers]

    # Split into AREA blocks
    blocks = re.split(r"AREA:", text)[1:]  # skip before first AREA

    results = []

    for block in blocks:
        # Find nearest header before this block
        block_start = text.find(block)
        region = None
        for pos, header in reversed(header_positions):
            if pos < block_start:
                region = header
                break

        # Extract area name
        area_match = re.match(r"\s*([^D]+?)\s+DATE:", block)
        area = area_match.group(1).strip() if area_match else None

        # Extract date
        date_match = re.search(r"DATE:\s*([A-Za-z]+\s+\d{2}\.\d{2}\.\d{4})", block)
        date = date_match.group(1) if date_match else None

        # Extract time
        time_match = re.search(r"TIME[: ]+\s*([\d\.A.MP\-\s]+)", block)
        time = time_match.group(1).strip() if time_match else None

        # Extract locations (everything after TIME)
        locations_match = re.search(r"TIME[: ]+.*?\s+(.+)", block)
        locations_raw = locations_match.group(1) if locations_match else ""
        locations = [loc.strip() for loc in locations_raw.split(",") if loc.strip()]

        results.append({
            "region": region,
            "area": area,
            "date": date,
            "time": time,
            "locations": locations
        })

    return results


# Example usage
raw_text = text
parsed = parse_schedule(raw_text)

# Save JSON
with open("kplc_schedule.json", "w") as f:
    json.dump(parsed, f, indent=4)

# Save CSV
with open("kplc_schedule.csv", "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=["region", "area", "date", "time", "locations"])
    writer.writeheader()
    for row in parsed:
        writer.writerow({
            "region": row["region"],
            "area": row["area"],
            "date": row["date"],
            "time": row["time"],
            "locations": "; ".join(row["locations"])
        })


In [9]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 64.8 MB/s eta 0:00:00


In [10]:
import pdfplumber
import re
import pandas as pd

def clean_text(text):
    text = text.replace("â€“", "–").replace("â€™", "'")
    text = re.sub(r"\s+", " ", text)
    return text.strip()

def parse_pdf(pdf_path):
    rows = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = clean_text(page.extract_text())

            # Split on 'DATE:' to chunk schedule entries
            entries = re.split(r"(?=DATE:)", text)
            for entry in entries:
                if not entry.strip():
                    continue

                # Date
                date_match = re.search(r"(Sunday|Monday|Tuesday|Wednesday|Thursday|Friday|Saturday)\s+\d{2}\.\d{2}\.\d{4}", entry)
                date = date_match.group(0) if date_match else ""

                # Start & End times
                time_match = re.search(r"(\d{1,2}\.\d{2}\s*(?:A\.M\.|P\.M\.))\s*–\s*(\d{1,2}\.\d{2}\s*(?:A\.M\.|P\.M\.))", entry)
                start_time, end_time = time_match.groups() if time_match else ("", "")

                # Region / Area
                region_match = re.search(r"(COUNTY AREA:|REGION AREA:|PARTS OF [A-Z]+|COUNTY [A-Za-z ]+)", entry)
                region = region_match.group(0) if region_match else ""

                # Locations (everything after the end time)
                if time_match:
                    locations = entry[time_match.end():].strip()
                else:
                    locations = entry.strip()

                rows.append([region, date, start_time, end_time, locations])

    df = pd.DataFrame(rows, columns=["region", "date", "start_time", "end_time", "locations"])
    return df

df = parse_pdf("/kaggle/working/kplc_pdfs/01JA7Y3RMRZVR9NNYMC4VRH4ZN.pdf")
df.to_csv("power_schedule_clean.csv", index=False)
print(df.head(10))


             region                  date  start_time   end_time  \
0   PARTS OF KISUMU                                                
1      REGION AREA:     Sunday 13.10.2024   9.00 A.M.  3.00 P.M.   
2                       Monday 14.10.2024  10.00 A.M.  4.00 P.M.   
3                      Tuesday 15.10.2024   9.00 A.M.  5.00 P.M.   
4                      Tuesday 15.10.2024                          
5  PARTS OF BUNGOMA  Wednesday 16.10.2024   9.00 A.M.  5.00 P.M.   
6                      Tuesday 15.10.2024   9.00 A.M.  5.00 P.M.   
7                      Tuesday 15.10.2024   8.00 A.M.  4.00 P.M.   
8   PARTS OF MIGORI  Wednesday 16.10.2024                          
9                    Wednesday 16.10.2024                          

                                           locations  
0  Power Maintenance Notice WESTERN REGION Notice...  
1  customers or to replace power lines during roa...  
2  AREA: PART OF MATHARE Kibos Prison, Cibol Indu...  
3  Kajulu Est, Kindo, Kianja, O

In [11]:
df.columns

Index(['region', 'date', 'start_time', 'end_time', 'locations'], dtype='object')

In [12]:
import pdfplumber

with pdfplumber.open("/kaggle/working/kplc_pdfs/01JA7Y3RMRZVR9NNYMC4VRH4ZN.pdf") as pdf:
    print(f"Number of pages: {len(pdf.pages)}")
    
    # Extract text from all pages
    for i, page in enumerate(pdf.pages):
        text = page.extract_text()
        print(f"\n--- Page {i+1} ---\n{text}")


Number of pages: 2

--- Page 1 ---
Power Maintenance Notice WESTERN REGION
Notice is hereby given under Rule 27 of the Electric Power Rules
PARTS OF KISUMU COUNTY
That the electricity supply will be interrupted as here under:
(It is necessary to interrupt supply periodically in order to facilitate AREA: KEDA CERAMICS
maintenance and upgrade of power lines to the network; to connect new DATE: Sunday 13.10.2024 TIME: 9.00 A.M. – 3.00 P.M.
customers or to replace power lines during road construction, etc.) Chepsweta, Kibigori Junction, KEDA Ceramics, Miwani, Jagir Sing & adjacent
customers.
NAIROBI REGION
AREA: CHIGA, KIBOS SUGAR RESEARCH
DATE: Monday 14.10.2024 TIME: 10.00 A.M. – 4.00 P.M.
AREA: PART OF MATHARE Kibos Prison, Cibol Industries, White Coal, Kibos Sugar Research, Properties
DATE: Tuesday 15.10.2024 TIME: 9.00 A.M. – 5.00 P.M. Kajulu Est, Kindo, Kianja, Okoko, Kajulu Waters, Bungu, Pifo Boys, Kasram, Wildo,
Mathare Area 1, Drive Inn Pri Sch, Shell Petrol Stn, Naivas S/Mkt, Al

In [13]:
import pdfplumber
import re
import json
import csv
from pathlib import Path

# -------- CONFIG --------
PDF_FILE = "/kaggle/working/kplc_pdfs/01J1FA2JXK3QNX0PM6YAP0P3T2.pdf"   # change to your PDF filename
JSON_FILE = "01J1FA2JXK3QNX0PM6YAP0P3T2.json"
CSV_FILE = "01J1FA2JXK3QNX0PM6YAP0P3T2.csv"

# -------- REGEX PATTERNS --------
region_pattern = re.compile(r"(.*REGION)")
county_pattern = re.compile(r"PARTS OF ([A-Z ]+) COUNTY")
area_pattern = re.compile(r"AREA:\s*(.*)")
date_time_pattern = re.compile(r"DATE:\s*([A-Za-z]+\s*\d{1,2}\.\d{1,2}\.\d{4})\s*TIME:\s*([\d.:APM\s–-]+)")

def parse_pdf(pdf_file):
    records = []
    current_region = None
    current_county = None
    current_area = None
    current_date = None
    current_time = None
    current_customers = []

    with pdfplumber.open(pdf_file) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if not text:
                continue

            lines = text.split("\n")
            for line in lines:
                line = line.strip()

                # Detect region
                if region_pattern.match(line):
                    current_region = line.strip()
                    current_county = None  # reset
                    continue

                # Detect county
                county_match = county_pattern.search(line)
                if county_match:
                    current_county = county_match.group(1).title().strip()
                    continue

                # Detect area
                area_match = area_pattern.search(line)
                if area_match:
                    # Save previous notice if exists
                    if current_area:
                        records.append({
                            "region": current_region,
                            "county": current_county,
                            "area": current_area,
                            "date": current_date,
                            "time": current_time,
                            "customers": [c.strip() for c in current_customers if c.strip()]
                        })
                    # Start new notice
                    current_area = area_match.group(1).strip()
                    current_date, current_time = None, None
                    current_customers = []
                    continue

                # Detect date & time
                dt_match = date_time_pattern.search(line)
                if dt_match:
                    current_date = dt_match.group(1).strip()
                    current_time = dt_match.group(2).replace("–", "-").strip()
                    continue

                # Customers (long lists)
                if current_area and not line.startswith("DATE:") and not line.startswith("AREA:"):
                    current_customers.extend([c.strip() for c in line.split(",")])

    # Save last block
    if current_area:
        records.append({
            "region": current_region,
            "county": current_county,
            "area": current_area,
            "date": current_date,
            "time": current_time,
            "customers": [c.strip() for c in current_customers if c.strip()]
        })

    return records

def save_json(data, filename):
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)

def save_csv(data, filename):
    keys = ["region", "county", "area", "date", "time", "customers"]
    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=keys)
        writer.writeheader()
        for row in data:
            writer.writerow({
                "region": row["region"],
                "county": row["county"],
                "area": row["area"],
                "date": row["date"],
                "time": row["time"],
                "customers": "; ".join(row["customers"])  # flatten list
            })

# -------- RUN --------
records = parse_pdf(PDF_FILE)
print(f"Extracted {len(records)} notices")

save_json(records, JSON_FILE)
save_csv(records, CSV_FILE)
print(f"Saved {JSON_FILE} and {CSV_FILE}")


Extracted 26 notices
Saved 01J1FA2JXK3QNX0PM6YAP0P3T2.json and 01J1FA2JXK3QNX0PM6YAP0P3T2.csv
